In [1]:
import pandas as pd

# Importing data from each year and creating one large data file

#### Looking at what is in each file using 2012 as a baseline in order to decide what to merge and how

In [2]:
Incident_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/2012/2012_Incident.csv')
Incident_2012.head()

,Complaint_No,Block_No,Direction,Street_Name,Street_Type,Suffix,Apt_No,City,State,Zipcode,...,Incident_to_date,Incident_to_Time,Place1,Place2,Clearance_Status,Clearance_Date,Case_Status,Reporting_Agency,Follow_up_Section,NIBRS_Hi_Class
0,20120616144301,9108,,REAMES,RD,,,CHARLOTTE,NC,28216,...,6/16/12,1330.0,Residential,Residential Care Facility ...,Exceptionally Cleared - Located (Missing Perso...,6/16/12,Close/Cleared,Charlotte-Mecklenburg Police Department,70-FIB - MISSING PERSONS ...,Missing Person ...
1,20120616144400,2903,N,DAVIDSON,ST,,,CHARLOTTE,NC,28205,...,6/16/12,1230.0,Residential,Apartment/Duplex Private Res ...,Open ...,,Further Investigation,Charlotte-Mecklenburg Police Department,06-EASTWAY DIVISION ...,Theft of Motor Vehicle Parts from Vehicle ...
2,20120616144700,601,W,24TH,ST,,,CHARLOTTE,NC,28206,...,NaN,NaN,Commercial Place,Other - Commercial Place ...,Open ...,,Inactive,Charlotte-Mecklenburg Police Department,02-METRO DIVISION ...,All Other Thefts ...
3,20120616145104,5522,,LABORDE,AV,,,CHARLOTTE,NC,28269,...,6/15/12,1635.0,Residential,Private Residence ...,Open ...,,Inactive,Charlotte-Mecklenburg Police Department,11-NORTH DIVISION ...,Damage/Vandalism Of Property ...
4,20120616145201,7718,,PRINDLE LAKE,DR,,,CHARLOTTE,NC,28227,...,6/16/12,1441.0,Residential,Private Residence ...,Exceptionally Cleared - Victim Chose not to Pr...,6/18/12,Close/Cleared,Charlotte-Mecklenburg Police Department,12-HICKORY GROVE DIVISION ...,Intimidation ...


In [3]:
Offenses_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/2012/2012_Offenses.csv')
Offenses_2012.head()

,Complaint_No,off_Attempt_Complete,Off_nibrs_crime_category,off_nibrs_classification,off_noc_code
0,20120616144301,Completed,Missing Person / Missing Person ...,Missing Person ...,800
1,20120616144400,Completed,Larceny / Larceny: Misdemeanor ...,Theft of Motor Vehicle Parts from Vehicle ...,23G
2,20120616144700,Completed,Damage to property / Injury to Real or Persona...,All Other Thefts ...,290
3,20120616144700,Completed,Larceny / Larceny: Misdemeanor ...,All Other Thefts ...,23H
4,20120616145104,Completed,Damage to property / Injury to Real or Persona...,Damage/Vandalism Of Property ...,290


In [26]:
print(Offenses_2012['Off_nibrs_crime_category'].nunique())
print(Offenses_2012['off_nibrs_classification'].nunique())
print(Incident_2012['NIBRS_Hi_Class'].nunique())

119
64
64


Offenses has an additional level of crime classification category with Off_nibrs_crime_category, while Incidents only has nibrs_hi_class. Will keep both because we plan to self-define property crimes.

In [4]:
Property_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/2012/2012_Property.csv')
Property_2012.head()

,Complaint_No,Property_No,Property_type,Property_Desc
0,20120616144400,1,Vehicle Tag ...,Vehicles and Parts ...
1,20120616144700,1,Other ...,Miscellaneous ...
2,20120616144700,2,Other ...,Miscellaneous ...
3,20120616145104,1,Dwelling: Single-Family ...,Structures ...
4,20120616145203,1,Purses/Handbags/Wallets ...,Money and Securities ...


In [27]:
Property_2012['Property_No'].describe()

count    203577.000000
mean          3.440271
std           6.246846
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max         192.000000
Name: Property_No, dtype: float64

Some crimes have a lot of impacted property items, with the average being 3.44, and the maximum being 192. Since we are focused There may be multiple property instances per crime if more than one thing was damaged or taken -- how do we want to handle this?

In [28]:
Property_2012['Property_Desc'].unique()

array(['Vehicles and Parts                                ',
       'Miscellaneous                                     ',
       'Structures                                        ',
       'Money and Securities                              ',
       'Other Property                                    ',
       'Withheld                                          ',
       'Electronic Equipment and Accessories              ',
       'Jewelry and Collectibles                          ',
       'Consumable Goods                                  ',
       'Building Materials                                ',
       'Electronic Equipment and Accesssories             ',
       'Crops and Landscaping                             ',
       'Animals                                           '], dtype=object)

In [30]:
Property_2012['Property_type'].unique()

array(['Vehicle Tag                                       ',
       'Other                                             ',
       'Dwelling: Single-Family                           ',
       'Purses/Handbags/Wallets                           ',
       'Credit/Debit Cards                                ',
       'Money/Cash                                        ',
       'Windows                                           ',
       'Withheld                                          ',
       'Telephone Equipment                               ',
       'Tools (Power and Hand)                            ',
       'TV Equipment                                      ',
       'Video Game Systems and Accessories                ',
       'Computer Hardware/Software                        ',
       'Camera/Optical Equipment                          ',
       'Doors/Garage Doors                                ',
       'Jewelry                                           ',
       'Vehicle Parts/Ac

Property_desc is a higher level category of property involved, while Property_type gets much more specific

In [5]:
Vehicle_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/2012/2012_Stolen_Vehicle.csv')
Vehicle_2012.head()

,Complaint_No,Vehicle_Year,Vechicle_Make,Vehicle_Model,Vehicle_Color,Vehicle_Body
0,20120616212404,2004,Buick ...,CENTURY,Silver/Silver ...,4-Door ...
1,20120617102701,1997,Chrysler ...,TOWN AND COUNTRY,White/White ...,Van ...
2,20120617153800,1999,Chevrolet ...,CAMARO,Green/Green ...,2-Door ...
3,20120617213000,1999,Oldsmobile ...,CUTLASS,White/White ...,4-Door ...
4,20120617215500,2006,Hyundai ...,SONATA,Tan/Tan ...,4-Door ...


Vehicles are only for stolen vehicles -- do we want to include these and consider them a type of property? Currently not included in property file (above) because they are considered a unique class.

In [6]:
Business_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/2012/2012_Victim_Business.csv')
Business_2012.head()

,Complaint_No,Victim_Bus_No,Victim_Bus_LegalName,Victim_Bus_Name,Victim_Bus_Type,Victim_Bus_Block_No,Victim_Bus_Direction,Victim_Bus_Street_Name,Victim_Bus_Suffix,Victim_Bus_Apt_No1,Victim_Bus_Apt_No2,Victim_Bus_City,Victim_Bus_State,Victim_Bus_ZipCode
0,20120616144700,1,COMPRESSOR SERVICE CO. ...,COMPRESSOR SERVICE CO. ...,Business ...,601,W,24TH,ST,,,CHARLOTTE,NC,28206
1,20120616151205,1,THE UNIVERSAL CHURCH ...,THE UNIVERSAL CHURCH ...,Religious Organization ...,5000,N,TRYON,ST,,,CHARLOTTE,NC,
2,20120616155402,1,STATE OF NORTH CAROLINA ...,STATE OF NORTH CAROLINA ...,Society/Public ...,,,,,,,,,
3,20120616160600,1,CITGO TIMES INC ...,CITGO TIMES INC ...,Business ...,8400,,OLD STATESVILLE,RD,,,CHARLOTTE,NC,28269
4,,2,STATE OF NORTH CAROLINA ...,STATE OF NORTH CAROLINA ...,Society/Public ...,,,,,,,,,


In [17]:
Business_2012['Victim_Bus_Type'].nunique()

129

In [16]:
Incident_2012['Place2'].nunique()

68

In [18]:
Incident_2012['Place1'].nunique()

5

Business includes the name of the business impacted (or State of NC for public crimes) as well as a more specific classification of business type with 129 unique values for Victim_Bus_Type rather than the 68 in Place2 of Incident.

In [19]:
Person_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/2012/2012_Victim_Person.csv')
Person_2012.head()

,Complaint_No,Victim_No,Victim_Age,Victim_Race,Victim_Gender,Victim_Ethnicity
0,20120616144301,1,50,Black,M,Other/Not Listed ...
1,20120616144400,1,26,White,F,Unknown/Refused ...
2,20120616145104,1,35,Black,F,Other/Not Listed ...
3,20120616145201,1,26,Black,M,Unknown/Refused ...
4,20120616145203,1,62,White,F,Other/Not Listed ...


Victim is personal information about the victim. Will not be needed.

In [20]:
Weapon_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/2012/2012_Weapons.csv')
Weapon_2012.head()

,Complaint_No,Weapon_Type
0,20120616155200,"Personal Weapons (hands,feet,teeth,etc.)"
1,20120616163800,"Personal Weapons (hands,feet,teeth,etc.)"
2,20120616163800,Other Weapon
3,20120616165800,"Personal Weapons (hands,feet,teeth,etc.)"
4,20120616170401,"Personal Weapons (hands,feet,teeth,etc.)"


Weapon file includes information about weapons will not be needed.

Final merging plan: Keep all information from incident to start, then add in Off_nibrs_crime_category from Offenses. Because this is about property, we will merge left with all property information on crime_no, leaving us with a row for each item impacted by a crime.

If we want to include vehicles, we can also concat all of the vehicle thefts and their relavant incidents, creating a new Property_Desc and Property_type of Vehicle (noting here that automobile is already a category in Property_type should be investigated)

For Victime_Bus, pull in business names as well as the more specific Victime_Bus_Type

Do not pull in anything from victim or weapon.